# Embedding - Traduction machine naive

Dans cet exercice, vous allez mettre en place votre premier système de traduction automatique!

In [ ]:
import pdb
import pickle
import string

import time

import matplotlib.pyplot as plt
import numpy as np
import scipy
import sklearn

from utils import get_dict

import warnings
warnings.filterwarnings('ignore')

<a name="1"></a>

# 1. Embeddings pour mots en Anglais et en Français

#### Le sous-emsemble de données

Pour réaliser cet exercice, nous utiliserons une sous-ensemble de word embeddings.

In [ ]:
en_embeddings_subset = pickle.load(open("en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("fr_embeddings.p", "rb"))

#### Consulter les données

* en_embeddings_subset: la clé (key) est un mot en anglais et la valeur est un
tableau de 300 dimensions, qui est l'embedding de ce mot.
```
'the': array([ 0.08007812,  0.10498047,  0.04980469,  0.0534668 , -0.06738281, ....
```

* fr_embeddings_subset: la clé (key) est un mot en français et la valeur est un
tableau de 300 dimensions, qui est l'embedding de ce mot.
```
'la': array([-6.18250e-03, -9.43867e-04, -8.82648e-03,  3.24623e-02,...
```

**Instruction**: Consultez les premiers éléments de `en_embeddings_subset` et `fr_embeddings_subset`

In [ ]:
### Code ici

### Propriétés d'Analogie des Embeddings

Avant d'attaquer la traduction automatique à l'aide des embeddings, ouvrons tout d'abord une petite parentèse afin de démontrer quelques propriétés des embeddings.

Vérifions ensemble que les embeddings de mots peuvent capturer des relations sémantiques complexes, permettant de réaliser des analogies comme "paris - france + italie = rome".


In [ ]:
def get_embedding(word, embeddings):
    """
    Récupère l'embedding d'un mot
    
    Args:
        word: Le mot dont on veut l'embedding
        embeddings: Dictionnaire des embeddings
        
    Returns:
        Le vecteur d'embedding du mot ou None si le mot n'est pas dans le dictionnaire
    """
    if word in embeddings:
        return embeddings[word]
    else:
        print(f"Le mot '{word}' n'est pas dans le vocabulaire.")
        return None
    
def analogy(word1, word2, word3, embeddings, n=5):
    """
    Résout une analogie de la forme: word1 est à word2 ce que ? est word3
    Exemple: paris est à france ce que ? est à italie -> word1=paris, word2=france, word3=italie
    
    Args:
        word1, word2, word3: Les trois mots de l'analogie
        embeddings: Dictionnaire des embeddings
        n: Nombre de résultats à retourner
        
    Returns:
        Liste des n mots les plus proches du résultat de l'analogie
    """
    # Récupérer les embeddings des mots
    emb1 = get_embedding(word1,embeddings)
    emb2 = get_embedding(word2,embeddings)
    emb3 = get_embedding(word3,embeddings)
    
    if emb1 is None or emb2 is None or emb3 is None:
        return []
    
    # Calculer le vecteur résultant de l'analogie: word1 - word2 + word3
    result_vector = emb1 - emb2 + emb3
    
    # Trouver les mots les plus proches du vecteur résultant
    # Normaliser le vecteur de référence
    vector_norm = result_vector / np.linalg.norm(result_vector)
    
    # Calculer la similarité cosinus avec tous les mots
    similarities = {}
    exclude=[word1, word2, word3]
    for word, emb in embeddings.items():
        if word in exclude:
            continue
        emb_norm = emb / np.linalg.norm(emb)
        similarity = np.dot(vector_norm, emb_norm)
        similarities[word] = similarity
    
    # Trier les mots par similarité décroissante
    sorted_words = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
     
    closest_words = sorted_words[:n]
    
    return closest_words


In [ ]:
print(analogy('france', 'paris', 'rome', fr_embeddings_subset))

# Tester l'analogie avec des mots en français
None

# Tester l'analogie avec des mots en anglais 
None

Fermons maintenant cette petit parenthèse et retournons à notre problème de Traduction Automatique :)

#### Charger les deux dictionnaires qui font correspondre les mots anglais aux mots français :
* Un dictionnaire d'entraînement
* Un dictionnaire de test

In [ ]:
# Chargement du dictionnaire Anglais -> Français
en_fr_train = get_dict('en-fr.train.txt')
print('The length of the English to French training dictionary is', len(en_fr_train))
en_fr_test = get_dict('en-fr.test.txt')
print('The length of the English to French test dictionary is', len(en_fr_train))

#### Consulter le dictionnaire Anglais -> Français

* `en_fr_train` est un dictionnaire où la clé est le mot anglais et la valeur
est la traduction française de ce mot anglais.
```
{'the': 'la',
 'and': 'et',
 'was': 'était',
 'for': 'pour',
```

* `en_fr_test` est similaire à `en_fr_train`, mais pour l'étape de test

In [ ]:
### Code ici


<a name="1-1"></a>

## 1.1 Embeddings et matrice de transformation

<a name="ex-01"></a>
#### Traduction du dictionnaire anglais -> français en utilisant des embeddings

Vous allez maintenant implémenter une fonction `get_matrices`, qui prend les données chargées
et renvoie les matrices `X` et `Y`.

Entrées :
- `en_fr` : Dictionnaire anglais -> français
- `en_embeddings` : Dictionnaire mots anglais vers les embeddings
- `fr_embeddings` : Dictionnaire mots français vers les embeddings

Renvoie :
- La matrice `X` et la matrice `Y`, où chaque ligne dans X est l'embedding de
  d'un mot anglais, et la même ligne dans Y est l'embedding de la version française
  de ce mot anglais.

<div style="width:image width px; font-size:100%; text-align:center;">
<img src='X_to_Y.jpg' alt="texte alternatif" width="largeur" height="hauteur" style="largeur:800px;hauteur:200px;" /> Figure 2 </div>

Utilisez le dictionnaire `en_fr` pour vous assurer que la i-ème ligne de la matrice `X`
correspond à la i-ème ligne de la matrice `Y`.

<a name="1-1"></a>

**Instructions**: Completez la function `get_matrices()`:
* Récupérez les embeddings des mots français et anglais

<details>
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
    <p>
        <ul>
            <li>Utilisez les dictionnaires `french_vecs` et `english_vecs`</li>
        </ul>
    </p>

In [ ]:
def get_matrices(en_fr, french_vecs, english_vecs):
    """
    Input:
        en_fr: English to French dictionary
        french_vecs: French words to their corresponding word embeddings.
        english_vecs: English words to their corresponding word embeddings.
    Output: 
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the projection matrix that minimizes the F norm ||X R -Y||^2.
    """


    # X_l et Y_l sont les listes d'embedding Anglais et Français
    X_l = list()
    Y_l = list()

    # Stocker les mots Anglais du dictionnaire des embeddings Anglais
    english_set = set(english_vecs.keys())

    # Stocker les mots français du dictionnaire des embeddings Français
    french_set = set(french_vecs.keys())

    # Stocker les mots français du dictionnaire Anglais-Français
    french_words = set(en_fr.values())

    # Boucle sur toutes les paires de mots anglais et français dans le dictionnaire anglais-français.

    for en_word, fr_word in en_fr.items():

        # Vérifie que le mot français a un embedding et que le mot anglais a un embedding.
        if fr_word in french_set and en_word in english_set:

            ### Code ici - Début
            # Recuperez l'embedding du mot anglais
            en_vec = None

            # Recuperez l'embedding du mot français
            fr_vec = None
            ### Code ici - Fin

            # Ajoute l'embedding anglais à la liste
            X_l.append(en_vec)

            # Ajoute l'embedding français à la liste
            Y_l.append(fr_vec)

    # Empile les vecteurs X_l dans une matrice X
    X = np.vstack(X_l)

    # Empile les vecteurs Y_l dans une matrice Y
    Y = np.vstack(Y_l)

    return X, Y


In [ ]:
# Récupére l'ensemble d'entrainement
X_train, Y_train = get_matrices(
    en_fr_train, fr_embeddings_subset, en_embeddings_subset)

<a name="2"></a>

# 2. Traduction

<div style="width:image width px; font-size:100%; text-align:center;"><img src='e_to_f.jpg' alt="alternate text" width="width" height="height" style="width:700px;height:200px;" /> Figure 1 </div>

<a name="2-1"></a>
## 2.1 Traduction comme transformation linéaire des embeddings

Étant donné des dictionnaires d'embedding de mots anglais et français, vous créerez une matrice de transformation `R`.
* Étant donné un embedding de mot anglais, $\mathbf{e}$, vous pouvez le multiplier par $\mathbf{eR}$ pour obtenir un nouvel embedding de mot $\mathbf{f}$.
    * À la fois $\mathbf{e}$ et $\mathbf{f}$ sont des [vecteurs ligne](https://en.wikipedia.org/wiki/Row_and_column_vectors).
* Vous pouvez ensuite calculer les voisins les plus proches de `f` dans les embeddings français et recommander le mot le plus similaire à l'embedding de mot transformé.

### Trouver une matrice `R` qui minimise l'équation suivante.

### Fonction de coût 

$$ \frac{1}{m} \|  \mathbf{X R} - \mathbf{Y} \|_{F}^{2}$$

où $m$ est le nombre d'exemples (nombre de lignes dans $\mathbf{X}$).

<a name="ex-02"></a>

#### Implémentation du mécanisme de traduction.

#### Étape 1 : Calcul du coût
* La fonction de coût :
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$

où $a_{i j}$ est la valeur dans la $i$-ème ligne et la $j$-ème colonne de la matrice $\mathbf{XR}-\mathbf{Y}$.

In [ ]:
def compute_loss(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        L: a matrix of dimension (m,n) - the value of the loss function for given X, Y and R.
    '''
    # m is the number of rows in X
    m = X.shape[0]
    
    # diff is XR - Y
    diff = np.dot(X,R)-Y

    # diff_squared is the element-wise square of the difference
    diff_squared = np.square(diff)

    # sum_diff_squared is the sum of the squared elements
    sum_diff_squared = np.sum(diff_squared)

    # loss i the sum_diff_squard divided by the number of examples (m)
    loss = sum_diff_squared/m
    return loss


### Étape 2 : Calcul du gradient du coût par rapport à la matrice de transformation R

* Calculer le gradient de la perte par rapport à la matrice de transformation `R`.
* Le gradient est une matrice qui indique dans quelle mesure un petit changement dans `R` affecte le changement de la fonction de perte.
* Le gradient nous donne la direction dans laquelle nous devons diminuer `R` pour minimiser la fonction de coût.
* $m$ est le nombre d'exemples d'entraînement (nombre de lignes dans $X$).
* La formule du gradient de la fonction de perte $𝐿(𝑋,𝑌,𝑅)$ est la suivante :

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$

In [ ]:
def compute_gradient(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        g: a matrix of dimension (n,n) - gradient of the loss function L for given X, Y and R.
    '''
    # m is the number of rows in X
    m = X.shape[0]

    # gradient is X^T(XR - Y) * 2/m
    gradient = np.dot(X.T,np.dot(X,R)-Y)*2/m
    return gradient


### Étape 3 : Recherche de la meilleure matrice R avec l'algorithme de descente de gradient

#### Descente de gradient

La [descente de gradient](https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html) est un algorithme itératif utilisé pour rechercher l'optimum d'une fonction. 
* Comme mentionné précédemment, le gradient de la perte par rapport à la matrice indique dans quelle mesure un petit changement dans une coordonnée de cette matrice affecte le changement de la fonction de perte.
* La descente de gradient utilise cette information pour changer de manière itérative la matrice `R` jusqu'à ce que nous atteignions un point où la perte est minimisée.

### Entraînement avec un nombre fixe d'itérations

La plupart du temps, nous itérons pour un nombre fixe d'étapes d'entraînement plutôt que d'itérer jusqu'à ce que la perte descende en dessous d'un seuil.

Pseudo-code :
1. Calculer le gradient $g$ de la perte par rapport à la matrice $R$.
2. Mettre à jour $R$ avec la formule :
$$R_{\text{new}}= R_{\text{old}}-\alpha g$$

Où $\alpha$ est le taux d'apprentissage, qui est un scalaire.

* Le taux d'apprentissage ou "pas" $\alpha$ est un coefficient qui décide combien nous voulons changer $R$ à chaque étape.
* Si nous changeons $R$ trop rapidement, nous pourrions passer à côté de l'optimum en prenant un pas trop grand.
* Si nous apportons seulement de petits changements à $R$, nous aurons besoin de nombreuses étapes pour atteindre l'optimum.
* Le taux d'apprentissage $\alpha$ est utilisé pour contrôler ces changements.
* Les valeurs de $\alpha$ sont choisies en fonction du problème, et nous utiliserons `learning_rate`$=0.0003$ comme valeur par défaut pour notre algorithme.

#### Instructions: Implementez `align_embeddings()`

<details>
<summary>
    <font size="3" color="darkgreen"><b>Aide</b></font>
</summary>
<p>
<ul>
    <li>Utilisez la fonction 'compute_gradient()' pour obtenir le gradient à chaque itération</li>

</ul>
</p>

In [ ]:
def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003):
    '''
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        train_steps: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    '''
    np.random.seed(129)

    # Nombre de lignes et de colonnes de R est égal au nombre de dimensions de l'embedding (e.g. 300)
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if i % 25 == 0:
            print(f"loss at iteration {i} is: {compute_loss(X, Y, R):.4f}")
        ### Code ici - Début
        # Utilisez la fonction implémentée plus haut et qui permet de calculer le gradient
        gradient = None

        # Mettre à jour R en soustrayant le taux d'apprentissage fois le gradient
        R = None
        ### Code ici - Fin
    return R


In [ ]:
# Tester l'implémentation
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = align_embeddings(X, Y)

**Expected Output:**
```
loss at iteration 0 is: 3.7242
loss at iteration 25 is: 3.6283
loss at iteration 50 is: 3.5350
loss at iteration 75 is: 3.4442
```

## Calculer la matrice de transformation R

Calculer la matrice de transformation R, en utilisant l'ensemble d'entraînement et en appelant la fonction align_embeddings().

REMARQUE : La cellule de code ci-dessous prendra quelques minutes pour s'exécuter complètement (~3 minutes)

In [ ]:
R_train = align_embeddings(X_train, Y_train, train_steps=400, learning_rate=0.8)

##### Expected Output

```
loss at iteration 0 is: 963.0146
loss at iteration 25 is: 46.9876
loss at iteration 50 is: 7.1546
loss at iteration 75 is: 1.8814
loss at iteration 100 is: 0.9121
loss at iteration 125 is: 0.6777
loss at iteration 150 is: 0.6070
loss at iteration 175 is: 0.5820
loss at iteration 200 is: 0.5723
loss at iteration 225 is: 0.5682
loss at iteration 250 is: 0.5663
loss at iteration 275 is: 0.5655
loss at iteration 300 is: 0.5651
loss at iteration 325 is: 0.5649
loss at iteration 350 is: 0.5648
loss at iteration 375 is: 0.5647
```

## 2.2 Tester la traduction

### Algorithme des k-plus proches voisins (k-NN)

[L'algorithme des k-plus proches voisins](https://fr.wikipedia.org/wiki/K-plus_proches_voisins)
* L'algorithme k-NN est une méthode qui prend un vecteur en entrée et trouve les autres vecteurs du jeu de données qui lui sont les plus proches.
* Le "k" représente le nombre de "plus proches voisins" à trouver (par exemple, k=2 trouve les deux voisins les plus proches).

### Recherche de l'embedding de la traduction
Puisque nous approximons la fonction de traduction des embeddings anglais vers les embeddings français par une matrice de transformation linéaire $\mathbf{R}$, la plupart du temps, nous n'obtiendrons pas l'embedding exacte d'un mot français lorsque nous transformons l'embedding $\mathbf{e}$ d'un mot anglais dans l'espace d'embedding français.
* C'est là que k-NN devient utile ! En utilisant le 1-NN avec $\mathbf{eR}$ comme entrée, nous pouvons rechercher un embedding $\mathbf{f}$ (sous forme de ligne) dans la matrice $\mathbf{Y}$ qui est la plus proche du vecteur transformé $\mathbf{eR}$.

### Similarité cosinus
La similarité cosinus entre les vecteurs $u$ et $v$ est calculée comme le cosinus de l'angle entre eux.
La formule est la suivante :

$$\cos(u,v) = \frac{u \cdot v}{\left\|u\right\| \left\|v\right\|}$$
* $\cos(u,v)$ = $1$ lorsque $u$ et $v$ se trouvent sur la même ligne et ont la même direction.
* $\cos(u,v)$ est $-1$ lorsqu'ils ont des directions exactement opposées.
* $\cos(u,v)$ est $0$ lorsque les vecteurs sont orthogonaux (perpendiculaires) les uns aux autres.

In [ ]:
def nearest_neighbor(v, candidates, k=1):
    """
    Input:
      - v, the vector you are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    similarity_l = []

    # for each candidate vector...
    for row in candidates:
        # get the cosine similarity
        cos_similarity = np.dot(v,row)/(np.linalg.norm(v)*np.linalg.norm(row))

        # append the similarity to the list
        similarity_l.append(cos_similarity)
     
    # sort the similarity list and get the indices of the sorted list
    sorted_ids = np.argsort(similarity_l)
    
    # get the indices of the k most similar candidate vectors
    k_idx = sorted_ids[-k:]
    
    return k_idx


In [ ]:
# Testez l'implementation du k-nn
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[nearest_neighbor(v, candidates, 3)])

**Expected Output**:

`[[9 9 9]
 [1 0 5]
 [2 0 1]]`

### Testez votre traduction et calculez sa précision

* Parcourez les embeddings de mots en anglais transformés et vérifiez si le vecteur de mot français le plus proche appartient au mot français qui est la traduction réelle.
* Obtenez un indice de l'embedding français le plus proche en utilisant `nearest_neighbor` (avec l'argument `k=1`), et comparez-le à l'indice de l'embedding en anglais que vous venez de transformer.
* Gardez une trace du nombre de fois où vous obtenez la traduction correcte.
* Calculez la précision comme suit : $$\text{précision} = \frac{\#(\text{prédictions correctes})}{\#(\text{prédictions totales})}$$

In [ ]:
def test_vocabulary(X, Y, R):
    '''
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    '''

    # The prediction is X times R
    pred = np.dot(X,R)

    # initialize the number correct to zero
    num_correct = 0

    # loop through each row in pred (each transformed embedding)
    for i in range(len(pred)):
        # get the index of the nearest neighbor of pred at row 'i'; also pass in the candidates in Y
        pred_idx = nearest_neighbor(pred[i], Y, k=1)

        # if the index of the nearest neighbor equals the row of i... \
        if pred_idx == i:
            # increment the number correct by 1.
            num_correct += 1

    # accuracy is the number correct divided by the number of rows in 'pred' (also number of rows in X)
    accuracy = num_correct/pred.shape[0]

    return accuracy


Quelle performance sur les données de test?

In [ ]:
X_val, Y_val = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [ ]:
acc = test_vocabulary(X_val, Y_val, R_train)  # this might take a minute or two
print(f"accuracy on test set is {acc:.3f}")

**Expected Output**:

```
0.561
```
### Résumé
Vous avez réussi à traduire des mots d'une langue à une autre sans les avoir jamais vus avec une précision de presque 56% en utilisant de l'algèbre linéaire de base et en apprenant une correspondance entre les mots d'une langue à une autre grâce aux embeddings!